In [10]:
import numpy as np
import cv2
from PIL import Image
import time


def bilinear_interpolation(x, y, image):
    x1, y1 = int(x), int(y)
    x2, y2 = x1 + 1, y1 + 1

    if x1 >= image.shape[1] - 1:
        x2 = x1
    if y1 >= image.shape[0] - 1:
        y2 = y1

    dx = x - x1
    dy = y - y1

    interpolated_value = (1 - dx) * (1 - dy) * image[y1, x1] + \
                         dx * (1 - dy) * image[y1, x2] + \
                         (1 - dx) * dy * image[y2, x1] + \
                         dx * dy * image[y2, x2]

    return interpolated_value

def rotate_image(image_path, output_path, angle, interpolation):
    # Load the grayscale image using OpenCV
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Get image dimensions
    height, width = image.shape

    # Convert angle to radians
    angle_rad = np.radians(angle)

    # Calculate the new dimensions of the rotated image to ensure it's fully visible
    new_width = int(np.abs(width * np.cos(angle_rad)) + np.abs(height * np.sin(angle_rad)))
    new_height = int(np.abs(width * np.sin(angle_rad)) + np.abs(height * np.cos(angle_rad)))

    # Create an empty canvas for the rotated image
    rotated_image = np.zeros((new_height, new_width), dtype=np.uint8)

    # Calculate the center of the original image
    center_x, center_y = width // 2, height // 2

    # Calculate the center of the rotated image
    new_center_x, new_center_y = new_width // 2, new_height // 2

    # Iterate through the rotated image and fill in pixel values
    for x in range(new_width):
        for y in range(new_height):
            # Calculate the corresponding coordinates in the original image
            x_original = int((x - new_center_x) * np.cos(-angle_rad) - (y - new_center_y) * np.sin(-angle_rad) + center_x)
            y_original = int((x - new_center_x) * np.sin(-angle_rad) + (y - new_center_y) * np.cos(-angle_rad) + center_y)

            # Check if the calculated coordinates are within the bounds of the original image
            if 0 <= x_original < width and 0 <= y_original < height and interpolation =='nearest':
                # Use nearest neighbor interpolation to fill in pixel values
                rotated_image[y, x] = image[y_original, x_original]
            elif 0 <= x_original < width and 0 <= y_original < height and interpolation =='bilinear':
                # Use bilinear interpolation to fill in pixel values
                rotated_image[y, x] = bilinear_interpolation(x_original, y_original, image)

    # Save the rotated image
    cv2.imwrite(output_path, rotated_image)
    
    return 1


# Function to rotate an image using Pillow's built-in bilinear interpolation
def rotate_image_builtin(image_path, output_path, angle):
    # Open the grayscale image using Pillow
    image = Image.open(image_path)

    # Rotate the image with bilinear interpolation
    rotated_image = image.rotate(angle, resample=Image.BILINEAR, expand=True)

    # Save the rotated image
    rotated_image.save(output_path)
    
    return 1


# Rotate 'box.png' by 5 degrees clockwise with nearest neighbor interpolation
start = time.time()
rotate_image('box.png', 'box_clockwise_5_nearest_custom.png', -5, interpolation='nearest')
end = time.time()
print(end-start)

# Rotate 'box.png' by 30 degrees counterclockwise with nearest neighbor interpolation
rotate_image('box.png', 'box_counterclockwise_30_nearest_custom.png', 30, interpolation='nearest')

# Rotate 'box.png' by 5 degrees clockwise with nearest neighbor interpolation
start = time.time()
rotate_image('box.png', 'box_clockwise_5_bilinear_custom.png', -5, interpolation='bilinear')
end = time.time()
print(end-start)

# Rotate 'box.png' by 30 degrees counterclockwise with nearest neighbor interpolation
rotate_image('box.png', 'box_counterclockwise_30_bilinear_custom.png', 30, interpolation='bilinear')

# Rotate 'box.png' by 5 degrees clockwise with built-in bilinear interpolation
rotate_image_builtin('box.png', 'box_clockwise_5_bilinear_builtin.png', -5)

# Rotate 'box.png' by 30 degrees counterclockwise with built-in bilinear interpolation
rotate_image_builtin('box.png', 'box_counterclockwise_30_bilinear_builtin.png', 30)



2.42446231842041
5.491029739379883


1